In [ ]:
"""
Create a prediction model with help of the PyTorch LSTM framework to forecast timeseries.

Source: https://www.geeksforgeeks.org/time-series-forecasting-using-pytorch/
"""

# Imports
import sys
sys.path.append('../')
sys.path.append('../../')
from imports import *

print(sys.path)

def saveAsJson (data:dict, name:str) -> None:
    with open(f"{name}.json", "w+") as outfile: 
        json.dump(data, outfile)


In [ ]:
# Create a IoT-Server object and grab the related timeseries from the InfluxDB database

# Inverter 2
inv2 = IotGrabber()
inv2.setDevices(["INV2", "TEMP", "HUM"])
inv2.setTimeAbsStart("2024-06-04T00:00:00Z")
inv2.setTimeAbsEnd  ("2024-09-12T23:59:00Z")
df_inv2 = inv2.get_df()
df_inv2.rename(columns=lambda x: x.replace('INV2', 'POWER'), inplace=True)

# Power measurement 'serverroom'
pm_serverroom = IotGrabber()
pm_serverroom.setDevices(["SHELLY_API_SERVERROOM_POWER"])
pm_serverroom.setTimeAbsStart("2024-07-16T00:00:00Z")
pm_serverroom.setTimeAbsEnd  ("2024-10-13T23:59:00Z")
df_serverroom = pm_serverroom.get_df()
df_serverroom.rename(columns=lambda x: x.replace('SHELLY_API_SERVERROOM_POWER', 'POWER'), inplace=True)

# Power measurement 'floor'
pm_floor = IotGrabber()
pm_floor.setDevices(["SHELLY_API_FLOOR_POWER"])
pm_floor.setTimeAbsStart("2024-07-16T00:00:00Z")
pm_floor.setTimeAbsEnd  ("2024-10-13T23:59:00Z")
df_floor = pm_floor.get_df()
df_floor.rename(columns=lambda x: x.replace('SHELLY_API_FLOOR_POWER', 'POWER'), inplace=True)


timeseries = [df_inv2, df_serverroom, df_floor]


In [ ]:
def generateLstmModel (df:pd.DataFrame,
                       name:str) -> torch:
    """Generate a LSTM model from a timeseries in a pd.Dataframe and save it"""

    # Generate data sets
    # Train test split
    training_data_len = math.ceil(len(df) * .9)

    # Splitting the dataset
    train_data = df[:training_data_len].iloc[:, :1]
    test_data = df[training_data_len:].iloc[:, :1]
    dataset_train = train_data.POWER.values
    dataset_train = np.reshape(dataset_train, (-1, 1)) # (-1,1) --> n-rows, 1 col
    # print(dataset_train.shape)

    # Selecting Open Price values
    dataset_test = test_data.POWER.values
    # Reshaping 1D to 2D array
    dataset_test = np.reshape(dataset_test, (-1, 1))
    # print(dataset_test.shape)

    scaler = MinMaxScaler(feature_range=(0, 1))
    # Scaling dataset
    scaled_train = scaler.fit_transform(dataset_train)
    # print(scaled_train[:5])

    # Normalizing values between 0 and 1
    scaled_test = scaler.transform(dataset_test)
    # print(scaled_test[:5])

    # Create sequences and labels for training data
    sequence_length = 50  # Number of time steps to look back
    X_train, y_train = [], []
    for i in range(len(scaled_train) - sequence_length):
        X_train.append(scaled_train[i:i + sequence_length])
        y_train.append(scaled_train[i + sequence_length])  # Predicting the value right after the sequence
    X_train, y_train = np.array(X_train), np.array(y_train)

    # Convert data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    # print(X_train.shape, y_train.shape)

    # Create sequences and labels for testing data
    sequence_length = 30  # Number of time steps to look back
    X_test, y_test = [], []
    for i in range(len(scaled_test) - sequence_length):
        X_test.append(scaled_test[i:i + sequence_length])
        y_test.append(scaled_test[i + sequence_length])  # Predicting the value right after the sequence
    X_test, y_test = np.array(X_test), np.array(y_test)

    # Convert data to PyTorch tensors
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32)

    # ---------------------------------------------------- #

    # Create the LSTM model
    class LSTMModel(nn.Module):
        def __init__(self, input_size, hidden_size, num_layers, dropout=0.2):
            super(LSTMModel, self).__init__()
            self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
            self.linear = nn.Linear(hidden_size, 1)

        def forward(self, x):
            out, _ = self.lstm(x)
            out = self.linear(out[:, -1, :])
            return out

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)

    input_size = 1
    num_layers = 3  # Increased number of layers (3)
    hidden_size = 128  # Increased number of hidden units (128)
    output_size = 1
    dropout = 0.2  # Added dropout for regularization

    model = LSTMModel(input_size, hidden_size, num_layers, dropout).to(device)
    loss_fn = nn.MSELoss(reduction='mean')
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0)  # Learning rate

    batch_size = 32*1  # Adjusted batch size (32)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    num_epochs = 10  # Increased number of epochs 100
    train_hist = []
    test_hist = []

    print("Start epochs...")
    for epoch in range(num_epochs):
        predictions_test_all = [] # added
        print(f"Epoche: {epoch+1}")
        total_loss = 0.0
        model.train()
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            predictions = model(batch_X)
            loss = loss_fn(predictions, batch_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        average_loss = total_loss / len(train_loader)
        train_hist.append(average_loss)

        model.eval()
        with torch.no_grad():
            total_test_loss = 0.0

            for batch_X_test, batch_y_test in test_loader:
                batch_X_test, batch_y_test = batch_X_test.to(device), batch_y_test.to(device)
                predictions_test = model(batch_X_test)
                predictions_test_cpu = predictions_test.cpu().numpy()[0, 0] # added
                predictions_test_all.append(predictions_test_cpu)           # added
                test_loss = loss_fn(predictions_test, batch_y_test)

                total_test_loss += test_loss.item()

            average_test_loss = total_test_loss / len(test_loader)
            test_hist.append(average_test_loss)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {average_loss:.4f}, Test Loss: {average_test_loss:.4f}')


            
            
    # Save model
    model_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    torch.save(model.state_dict(), f"{model_time}-{name}.model")

            
    print(f"Model generation done")      

    x = np.linspace(1,num_epochs,num_epochs)
    plt.plot(x,train_hist,scalex=True, label="Training loss")
    plt.plot(x, test_hist, label="Test loss")
    plt.legend()
    plt.show()


    # -------------------------------- #
    
    # Validate
    num_forecast_steps = 30*1 # (30)
    sequence_to_plot = X_test.squeeze().cpu().numpy()
    historical_data = sequence_to_plot[-1]

    forecasted_values = []
    model.eval()
    with torch.no_grad():
        for _ in range(num_forecast_steps):
            historical_data_tensor = torch.as_tensor(historical_data).view(1, -1, 1).float().to(device)
            predicted_value = model(historical_data_tensor).cpu().numpy()[0, 0]
            forecasted_values.append(predicted_value)
            historical_data = np.roll(historical_data, shift=-1)
            historical_data[-1] = predicted_value

    last_date = test_data.index[-1]
    future_dates = pd.date_range(start=last_date + pd.Timedelta(1, unit='m'), periods=num_forecast_steps, freq="1min") # add 1 min steps to the df

    forecasted = scaler.inverse_transform(np.array(forecasted_values).reshape(-1, 1)).flatten()
    predicted = pd.DataFrame()
    predicted.insert(0, "Date", future_dates)
    predicted.insert(1, "Predict", forecasted)
    predicted.set_index("Date", inplace=True)

    plt.rcParams['figure.figsize'] = [14, 4]

    # Plot complete data
    # plt.plot(df.index[:], df.POWER, label="Complete Data", color="b")

    # Plot training data
    # plt.plot(train_data.index[:], train_data[:], label="Training Data", color="g")

    # Plot test data
    # plt.plot(test_data.index[:], test_data[:], label="Test Data", color="r")

    # Plot predicted data
    # plt.plot(predicted.index[:], predicted[:], label='Predicted', color='g')

    # Misc Plots
    # plt.plot(test_data.index[-num_forecast_steps:], test_data[-num_forecast_steps:], label='actual values', color='green')
    # plt.plot(test_data.index[-1:].append(future_dates), np.concatenate([test_data[-1:], scaler.inverse_transform(np.array(forecasted_values).reshape(-1, 1)).flatten()]), label='forecasted values', color='red')


    # plt.xlabel('Time Step')
    # plt.ylabel('Value')
    # plt.legend()
    # plt.title('Time Series Forecasting')
    # plt.grid(True)
    # plt.ylim(-100,6000)
    # plt.show()

    # Evaluate the model and calculate RMSE and R² score
    model.eval()
    with torch.no_grad():
        test_predictions = []
        for batch_X_test in X_test:
            batch_X_test = batch_X_test.to(device).unsqueeze(0)  # Add batch dimension
            test_predictions.append(model(batch_X_test).cpu().numpy().flatten()[0])

    test_predictions = np.array(test_predictions)

    # Calculate RMSE and R² score
    rmse = np.sqrt(mean_squared_error(y_test.cpu().numpy(), test_predictions))
    r2 = r2_score(y_test.cpu().numpy(), test_predictions)

    # print(f'RMSE: {rmse:.4f}')
    # print(f'R² Score: {r2:.4f}')

    test_predictions_all = scaler.inverse_transform(np.array(test_predictions).reshape(-1, 1)).flatten()
    # plt.plot(test_data.index[30:], test_data.POWER[30:], label="Test Data", color="b")
    # plt.plot(test_data.index[30:], test_predictions_all, label="Test Data_predicted", color='green')
    # plt.show()

    dPred = test_data.POWER[30:]- test_predictions_all

    results = {"test_data": test_data,
               "test_prediction_data": test_predictions_all,
               "predicted": predicted,
               "num_forecast_steps": num_forecast_steps,
               "dPred": dPred,
               "rmse": rmse,
               "r2": r2
               }

    return model, results

In [ ]:
inv2_model, inv2_lstm_results = generateLstmModel(df_inv2, "INV2")
serverroom_model, serverroom_results = generateLstmModel(df_serverroom, "Serverroom")
floor_model, floor_results = generateLstmModel(df_floor, "Floor")

In [ ]:
# Inverter2
errors = {"rmse": str(inv2_lstm_results["rmse"]), "r2": str(inv2_lstm_results["r2"])}
saveAsJson(errors, "LSTM-Inverter2")
test_data = inv2_lstm_results["test_data"]
test_prediction_data = inv2_lstm_results["test_prediction_data"]

plt.plot(test_data.index[30:], test_data.POWER[30:], label="Test Data", color="b")
plt.plot(test_data.index[30:], test_prediction_data, label="Test Data_predicted", color='green')
plt.show()

# Serverroom
errors = {"rmse": str(serverroom_results["rmse"]), "r2": str(serverroom_results["r2"])}
saveAsJson(errors, "LSTM-Serverroom")
test_data = serverroom_results["test_data"]
test_prediction_data = serverroom_results["test_prediction_data"]

plt.plot(test_data.index[30:], test_data.POWER[30:], label="Test Data", color="b")
plt.plot(test_data.index[30:], test_prediction_data, label="Test Data_predicted", color='green')
plt.show()

# Floor
errors = {"rmse": str(floor_results["rmse"]), "r2": str(floor_results["r2"])}
saveAsJson(errors, "LSTM-Floor")
test_data = floor_results["test_data"]
test_prediction_data = floor_results["test_prediction_data"]

plt.plot(test_data.index[30:], test_data.POWER[30:], label="Test Data", color="b")
plt.plot(test_data.index[30:], test_prediction_data, label="Test Data_predicted", color='green')
plt.show()

In [ ]:
df_test_prediction = pd.DataFrame({"Date":test_data.index[30:],
                                   "Test":test_data.POWER[30:],
                                   "Predict":test_prediction_data,
                                   "error":test_data.POWER[30:]-test_prediction_data})

df_test_prediction.set_index("Date", inplace=True)
df_test_prediction.to_csv("LSTM-Serverroom.csv", sep=";")

In [ ]:
df_test_prediction.plot()